In [ ]:
# boas práticas para programação em produção:
# 0. utilizar arquivo .py em vez de notebooks; importar todas as bibliotecas no início do código;
# 1. criar funções separadas para cada ação que queremos executar;
# 2. determinar os tipos de variáveis que as funções devem receber;
# 3. reaproveitar o código ao máximo, evitar repetições;
# 4. utilizar try/except para deixar mais profissional, exibindo erro f'Erro: {e}'
# 5. em vez de print, utilizar biblioteca/módulo logging para exibir mensagens durante execução;

# OBS: aqui, faremos uma versão para teste; depois, o código será refatorado seguindo os princípios acima, para ficar mais profissional

In [1]:
import pandas as pd
import requests, logging

from datetime import datetime as dt

In [ ]:
# o problema consiste em:
# 1. extrair json (dic) com os dados da API referente a um produto específico, que vamos escolher;
# 2. ao obter os dados, vamos criar um dataframe com as colunas que nos interessam;
# 3. dentre as colunas que nos interessam, vamos verificar a necessidade de fazer alguma transformação;
# 4. tudo estando redondo, vamos salvar as informações em um arquivo csv.

In [33]:
# parte 1 - função para extrair as informações do site a partir do código do produto (str)

def extraindo_infos(cod_produto:str):
    
    url = 'https://world.openfoodfacts.org/api/v2/product/'
    url_completa = url + cod_produto

    resposta = requests.get(url_completa)

    dados_brutos = resposta.json()

    return dados_brutos

In [45]:
url_completa

'https://world.openfoodfacts.org/api/v2/product/7891000369371'

In [34]:
cod_produto = '7891000369371'

In [ ]:
extraindo_infos(cod_produto)

In [30]:
# criando um dataframe com as colunas de nosso interesse: id do produto, nome do produto, 

dados_brutos = resposta.json() # note que isso é o que a nossa função extraindo_infos retorna!

dados_produto = dados_brutos.get('product', None)

In [67]:
dados2 = pd.json_normalize(dados_produto)
dados2

,_id,_keywords,added_countries_tags,allergens,allergens_from_ingredients,allergens_from_user,allergens_hierarchy,allergens_tags,brands,brands_tags,...,nutriscore.2023.grade,nutriscore.2023.nutrients_available,nutriscore.2023.nutriscore_applicable,nutriscore.2023.nutriscore_computed,selected_images.front.display.es,selected_images.front.display.pt,selected_images.front.small.es,selected_images.front.small.pt,selected_images.front.thumb.es,selected_images.front.thumb.pt
0,7891000369371,"[80g, ao, chocolate, crunch, leite, pacote]",[],,,(pt),[],[],CRUNCH,[crunch],...,unknown,0,0,0,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...


In [31]:
# agora, vamos utilizar um recurso do pandas para trabalhar com json e deixar em formato bidimensional (LXC): json_normalize

dados = pd.json_normalize(dados_brutos)
dados

,code,status,status_verbose,product._id,product._keywords,product.added_countries_tags,product.allergens,product.allergens_from_ingredients,product.allergens_from_user,product.allergens_hierarchy,...,product.states,product.states_hierarchy,product.states_tags,product.traces,product.traces_from_ingredients,product.traces_from_user,product.traces_hierarchy,product.traces_tags,product.unknown_nutrients_tags,product.weighers_tags
0,7891000369371,1,product found,7891000369371,"[80g, ao, chocolate, crunch, leite, pacote]",[],,,(pt),[],...,"en:to-be-completed, en:nutrition-facts-complet...","[en:to-be-completed, en:nutrition-facts-comple...","[en:to-be-completed, en:nutrition-facts-comple...",,,(pt),[],[],[],[]


In [69]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 437 entries, _id to selected_images.front.thumb.pt
dtypes: float64(7), int64(283), object(147)
memory usage: 3.5+ KB


In [68]:
for i in dados2.columns:
    print(i)

_id
_keywords
added_countries_tags
allergens
allergens_from_ingredients
allergens_from_user
allergens_hierarchy
allergens_tags
brands
brands_tags
categories_properties_tags
checkers_tags
code
codes_tags
complete
completeness
correctors_tags
countries
countries_hierarchy
countries_lc
countries_tags
created_t
creator
data_quality_bugs_tags
data_quality_errors_tags
data_quality_info_tags
data_quality_tags
data_quality_warnings_tags
ecoscore_grade
ecoscore_tags
editors_tags
entry_dates_tags
food_groups_tags
id
image_front_small_url
image_front_thumb_url
image_front_url
image_small_url
image_thumb_url
image_url
informers_tags
ingredients_lc
interface_version_created
interface_version_modified
lang
languages_hierarchy
languages_tags
last_edit_dates_tags
last_image_dates_tags
last_image_t
last_modified_t
lc
main_countries_tags
max_imgid
misc_tags
nova_group_debug
nova_group_error
nova_groups_tags
nutrient_levels_tags
nutriscore_2021_tags
nutriscore_2023_tags
nutriscore_grade
nutriscore_tags
n

In [51]:
dados.to_json('./dados_crunch_amostra.json')

In [66]:
# selecionando as colunas de nosso interesse: # image_front_small_url

dados_target = dados[['product._id', 'product.product_name', 'product.brands', 'product.image_front_small_url']]
dados_target

,product._id,product.product_name,product.brands,product.image_front_small_url
0,7891000369371,Chocolate Ao Leite Crunch Pacote 80g,CRUNCH,https://images.openfoodfacts.org/images/produc...


In [74]:
# parte 2 - criando um dataframe com as colunas de nosso interesse a partir dos dados brutos que obtivemos no requests.get 

def selecionando_dados(dados_brutos):
    dados_produto = dados_brutos.get('product', None)
    dados = pd.json_normalize(dados_produto) # transformando o json em dataframe, bidimensional (LXC)
    dados_target = dados[['_id', 'product_name', 'brands', 'image_front_small_url', 'created_t']] # selecionando colunas de nosso interesse

    return dados_target # aqui, retornamos um dataframe com as colunas que nos interessam


In [75]:
selecionando_dados(dados_brutos)

,_id,product_name,brands,image_front_small_url,created_t
0,7891000369371,Chocolate Ao Leite Crunch Pacote 80g,CRUNCH,https://images.openfoodfacts.org/images/produc...,1686621225


In [84]:
dados_target

,product._id,product.product_name,product.brands,product.image_front_small_url
0,7891000369371,Chocolate Ao Leite Crunch Pacote 80g,CRUNCH,https://images.openfoodfacts.org/images/produc...


In [82]:
dados_target['product.created_t'] = pd.to_datetime(dados_target['product.created_t'], units='s')

KeyError: 'product.created_t'

In [80]:
# parte 3 - agora, vamos incluir algumas colunas e ver se precisamos fazer alguma transformação nos dados que já obtivemos

def modelagem_final(dados_target):
    # primeiro: converter a coluna created_t para o formato datetime em data
    dados_target['created_t'] = pd.to_datetime(dados_target['created_t'], units='s')
    # depois: incluir coluna com a data de inclusão do registro no dataframe

    return(dados_target)

In [81]:
modelagem_final(dados_target)

KeyError: 'product.created_t'